In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Se carga el conjunto de datos
data = pd.read_csv('train.csv')

# Se rellenan las edades faltantes con la mediana
data['Age'].fillna(data['Age'].median(), inplace=True)

# Se rellenan los valores faltantes de la columna 'Embarked' con el valor más común
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

# Se eliminan las características irrelevantes
data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

# Se convierte la columna 'Sex' en una variable numérica (1 para 'male' y 0 para 'female')
data['Sex'] = data['Sex'].map({'male': 1, 'female': 0})

# Se convierte la columna 'Embarked' en variables numéricas utilizando la codificación One-Hot
embarked_encoded = pd.get_dummies(data['Embarked'], prefix='Embarked')
data = pd.concat([data, embarked_encoded], axis=1)
data.drop(['Embarked'], axis=1, inplace=True)

# Se guarda el conjunto de datos limpio en un nuevo archivo CSV
data.to_csv('titanic_data_cleaned.csv', index=False)

In [3]:
data = pd.read_csv('titanic_data_cleaned.csv')

# Separar las características (X) y la variable objetivo (y)
X = data.drop('Survived', axis=1)
y = data['Survived']

# Partición entrenamiento-prueba (70% para entrenamiento, 30% para prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Realizar el escalado de datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Entrenar el modelo de Regresión Logística con parámetros ajustados
clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de pruebas
y_pred = clf.predict(X_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy (Adjusted Parameters): {accuracy:.2f}')

conf_matrix = confusion_matrix(y_test, y_pred)
print(f'Confusion Matrix:\n{conf_matrix}')

class_report = classification_report(y_test, y_pred)
print(f'Classification Report:\n{class_report}')

Accuracy (Adjusted Parameters): 0.81
Confusion Matrix:
[[135  22]
 [ 30  81]]
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.86      0.84       157
           1       0.79      0.73      0.76       111

    accuracy                           0.81       268
   macro avg       0.80      0.79      0.80       268
weighted avg       0.81      0.81      0.80       268



In [4]:
# ARBOL DE DECISION

In [5]:
# Entrenamiento del modelo de Árbol de decisión
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predicciones en el conjunto de prueba
y_pred = clf.predict(X_test)

# Rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

conf_matrix = confusion_matrix(y_test, y_pred)
print(f'Confusion Matrix:\n{conf_matrix}')

class_report = classification_report(y_test, y_pred)
print(f'Classification Report:\n{class_report}')

Accuracy: 0.78
Confusion Matrix:
[[128  29]
 [ 31  80]]
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.82      0.81       157
           1       0.73      0.72      0.73       111

    accuracy                           0.78       268
   macro avg       0.77      0.77      0.77       268
weighted avg       0.78      0.78      0.78       268



In [6]:
# PARAMETROS MEJORADOS
# Entrene el modelo de Árbol de decisión con parámetros mejorados
clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=20, random_state=42)
clf.fit(X_train, y_train)

# Realice predicciones en el conjunto de prueba
y_pred = clf.predict(X_test)

# Rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

conf_matrix = confusion_matrix(y_test, y_pred)
print(f'Confusion Matrix:\n{conf_matrix}')

class_report = classification_report(y_test, y_pred)
print(f'Classification Report:\n{class_report}')

Accuracy: 0.82
Confusion Matrix:
[[143  14]
 [ 33  78]]
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.91      0.86       157
           1       0.85      0.70      0.77       111

    accuracy                           0.82       268
   macro avg       0.83      0.81      0.81       268
weighted avg       0.83      0.82      0.82       268



In [7]:
#BUSQUEDA DE HYPERPARAMETROS
# Defina el árbol de decisión base
base_dt = DecisionTreeClassifier(random_state=42)

# Defina los hiperparámetros que desea explorar en la búsqueda en cuadrícula
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

# Realice la búsqueda en cuadrícula
grid_search = GridSearchCV(base_dt, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Imprima los mejores hiperparámetros y la puntuación correspondiente
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Rendimiento del mejor modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

conf_matrix = confusion_matrix(y_test, y_pred)
print(f'Confusion Matrix:\n{conf_matrix}')

class_report = classification_report(y_test, y_pred)
print(f'Classification Report:\n{class_report}')

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2}
Best Score: 0.8073806451612905
Accuracy: 0.82
Confusion Matrix:
[[141  16]
 [ 33  78]]
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.90      0.85       157
           1       0.83      0.70      0.76       111

    accuracy                           0.82       268
   macro avg       0.82      0.80      0.81       268
weighted avg       0.82      0.82      0.81       268



In [8]:
# RANDOM FOREST

In [9]:
# Defina el Random Forest base
base_rf = RandomForestClassifier(random_state=42)

# Defina los hiperparámetros que desea explorar en la búsqueda en cuadrícula
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

# Realice la búsqueda en cuadrícula
grid_search = GridSearchCV(base_rf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Imprima los mejores hiperparámetros y la puntuación correspondiente
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Rendimiento del mejor modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

conf_matrix = confusion_matrix(y_test, y_pred)
print(f'Confusion Matrix:\n{conf_matrix}')

class_report = classification_report(y_test, y_pred)
print(f'Classification Report:\n{class_report}')

Fitting 5 folds for each of 360 candidates, totalling 1800 fits
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 10}
Best Score: 0.8346451612903227
Accuracy: 0.82
Confusion Matrix:
[[145  12]
 [ 37  74]]
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.92      0.86       157
           1       0.86      0.67      0.75       111

    accuracy                           0.82       268
   macro avg       0.83      0.80      0.80       268
weighted avg       0.82      0.82      0.81       268



In [10]:
# Cargar el conjunto de datos de prueba
test_data = pd.read_csv('test_kaggle.csv')

# Rellenar las edades faltantes con la mediana
test_data['Age'].fillna(data['Age'].median(), inplace=True)

# Eliminar las características irrelevantes
test_data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

# Convertir la columna 'Sex' en una variable numérica (1 para 'male' y 0 para 'female')
test_data['Sex'] = test_data['Sex'].map({'male': 1, 'female': 0})

# Verificar si la columna 'Embarked' existe en el conjunto de datos de prueba
if 'Embarked' in test_data.columns:
    # Convertir la columna 'Embarked' en variables numéricas utilizando la codificación One-Hot
    embarked_encoded = pd.get_dummies(test_data['Embarked'], prefix='Embarked')
    test_data = pd.concat([test_data, embarked_encoded], axis=1)
    test_data.drop(['Embarked'], axis=1, inplace=True)
    
# Eliminar la columna 'Unnamed: 0' si existe
if 'Unnamed: 0' in test_data.columns:
    test_data.drop(['Unnamed: 0'], axis=1, inplace=True)

# Rellenar los valores NaN con la mediana de la columna correspondiente
test_data.fillna(test_data.median(), inplace=True)

# Transformar los datos y hacer la predicción
test_data_scaled = scaler.transform(test_data)
test_predictions = best_model.predict(test_data_scaled)

# Crear un DataFrame para el resultado
result = pd.DataFrame()
result['PassengerId'] = test_data['PassengerId']
result['Survived'] = test_predictions

# Guardar el resultado en un archivo CSV
result.to_csv('test_predictions.csv', index=False)